# MIMIC-RiskLab: Interactive Demo

Welcome to the **MIMIC-RiskLab** interactive analysis demo. This notebook demonstrates the core diagnostic pipeline and visualization logic used to predict patient mortality risk based on clinical laboratory events.

### 🏥 Project Goal
Identify predictive laboratory trends and help clinicians prioritize high-risk patients using data-driven insights from the MIMIC-III database.

## 🛠️ Step 1: Environment Setup
First, we'll install necessary dependencies.

In [ ]:
# Install dependencies
!pip install pandas seaborn matplotlib scikit-learn

## 📝 Step 2: Synthetic Data Generation
Since the full MIMIC-III dataset requires specialized access, this demo generates **synthetic clinical data** that follows the project schema for demonstration purposes.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Set seed for reproducibility
np.random.seed(42)

# Generate 500 patients
n = 500
data = {
    'age': np.random.normal(65, 15, n).clip(18, 90),
    'gender': np.random.randint(0, 2, n),
    'lab_count': np.random.gamma(20, 2, n).astype(int),
    'admission_type_EMERGENCY': np.random.randint(0, 2, n)
}

# Create dependency: higher abnormal labs and lab counts correlate with higher mortality risk
data['abnormal_count'] = (data['lab_count'] * np.random.uniform(0.1, 0.4, n)).astype(int)
risk_score = (data['abnormal_count'] * 0.5) + (data['lab_count'] * 0.1) + (data['age'] * 0.05) + np.random.normal(0, 5, n)
data['hospital_expire_flag'] = (risk_score > np.percentile(risk_score, 80)).astype(int)

df = pd.DataFrame(data)
print("Synthetic Data Schema:")
print(df.head())
print(f"\nClass Balance (Mortality): {df['hospital_expire_flag'].mean():.2%}")

## 📊 Step 3: Predictive Analysis & Visualizations
We now replicate the analysis used in the main project to identify risk drivers.

In [ ]:
# 1. Feature Correlation Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Clinical Feature Correlation Matrix')
plt.show()

# 2. Abnormal Lab Counts vs Mortality (The Key Finding)
plt.figure(figsize=(8, 6))
sns.boxplot(x='hospital_expire_flag', y='abnormal_count', data=df)
plt.title('Abnormal Lab Counts vs Mortality Outcome')
plt.xlabel('Mortality (0=No, 1=Yes)')
plt.ylabel('Number of Abnormal Lab Results')
plt.show()

## 🤖 Step 4: Machine Learning
Training the Random Forest model to predict mortality risk.

In [ ]:
X = df.drop('hospital_expire_flag', axis=1)
y = df['hospital_expire_flag']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

importance = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importances_})
importance = importance.sort_values('importance', ascending=False)

plt.figure(figsize=(8, 5))
sns.barplot(x='importance', y='feature', data=importance)
plt.title('Feature Importance for Mortality Prediction')
plt.show()

---
**End of Demo**. This notebook illustrates the automated analysis pipeline of MIMIC-RiskLab. For local execution on full datasets, refer to the [GitHub Repository](https://github.com/bashirAI-lab/MIMIC-RiskLab).